In [1]:
import sys, geopandas as gpd, shapely
print("Python:", sys.executable)
print("geopandas:", gpd.__version__, "| shapely:", shapely.__version__)


Python: c:\Users\mmamani\Anaconda3\envs\selenium_env\python.exe
geopandas: 1.1.1 | shapely: 2.1.2


In [2]:
import geopandas as gpd
path = r"D:\Diplomado -\Python - intermedio\Tarea 2\shape_file\DISTRITOS.shp"

try:
    gdf_dist = gpd.read_file(path, engine="pyogrio")  # rápido
except Exception:
    gdf_dist = gpd.read_file(path, engine="fiona")    # fallback

gdf_dist = gdf_dist.to_crs("EPSG:4326") if gdf_dist.crs else gdf_dist.set_crs("EPSG:4326")
print(gdf_dist.crs, len(gdf_dist))


EPSG:4326 1873


In [ ]:
# -*- coding: utf-8 -*-
"""
Tarea 2 - Pipeline con GeoPandas (pyogrio)
Genera:
  1) gdf_hospitales_clean.parquet
  2) distritos_enriched.parquet
  3) deptos_enriched.parquet

Entradas:
  - D:\Diplomado -\Python - intermedio\Tarea 2\IPRESS.csv
  - D:\Diplomado -\Python - intermedio\Tarea 2\shape_file\DISTRITOS.shp
Requisitos:
  conda install -c conda-forge geopandas pyogrio shapely>=2.0 fastparquet
"""

import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# --------------------------------
BASE = r"D:\Diplomado -\Python - intermedio\Tarea 2"
HOSP_CSV = rf"{BASE}\IPRESS.csv"
DIST_SHP = rf"{BASE}\shape_file\DISTRITOS.shp"

OUT_HOSP_PARQ  = rf"{BASE}\gdf_hospitales_clean.parquet"     # tabular (sin geometry)
OUT_DIST_PARQ  = rf"{BASE}\distritos_enriched.parquet"       # tabular (sin geometry)
OUT_DEPTO_PARQ = rf"{BASE}\deptos_enriched.parquet"          # tabular (sin geometry)

# (opcional) si tienes pyarrow>=14 + pyarrow-geopandas, también deja GeoParquet:
OUT_HOSP_GEO_PARQ = rf"{BASE}\gdf_hospitales_clean_geoparquet.parquet"
OUT_DIST_GEO_PARQ = rf"{BASE}\distritos_enriched_geoparquet.parquet"

# Usar pyogrio por defecto (rápido y estable)
gpd.options.io_engine = "pyogrio"


def read_ipress_as_points() -> gpd.GeoDataFrame:
    # 1) Leer CSV con encodings típicos
    df = None
    for enc in ("latin1", "utf-8-sig", "utf-8"):
        try:
            df = pd.read_csv(HOSP_CSV, encoding=enc)
            print(f"✔ IPRESS leído con encoding: {enc}")
            break
        except Exception:
            pass
    if df is None:
        raise RuntimeError("No pude leer IPRESS.csv con latin1/utf-8-sig/utf-8")

    # 2) Asegurar columnas y convertir a numérico
    for c in ("ESTE", "NORTE"):
        if c not in df.columns:
            raise ValueError(f"Falta la columna '{c}' en IPRESS.csv")
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # 3) Filtrar rangos válidos (ESTE=lat, NORTE=lon) y deduplicar
    df = df[df["ESTE"].between(-90, 90) & df["NORTE"].between(-180, 180)].copy()
    id_cands = ["Código Único", "Codigo Único", "Codigo Unico", "Código Unico "]
    id_col = next((c for c in id_cands if c in df.columns), None)
    if id_col:
        before = len(df)
        df = df.drop_duplicates(subset=[id_col])
        print(f"✔ Deduplicado por '{id_col}': {before} -> {len(df)}")
    else:
        df = df.drop_duplicates()

    # 4) Crear geometría de puntos (EPSG:4326)
    geom = [Point(xy) for xy in zip(df["NORTE"], df["ESTE"])]
    gdf = gpd.GeoDataFrame(df, geometry=geom, crs="EPSG:4326")
    return gdf


def save_parquet_tabular(df_or_gdf: pd.DataFrame, out_path: str):
    """Escribe Parquet tabular con fastparquet (no requiere pyarrow)."""
    # Si viene un GeoDataFrame, quita la geometría
    if isinstance(df_or_gdf, gpd.GeoDataFrame) and "geometry" in df_or_gdf.columns:
        df_or_gdf = pd.DataFrame(df_or_gdf.drop(columns="geometry"))
    df_or_gdf.to_parquet(out_path, index=False, engine="fastparquet")
    print("✔ Parquet:", out_path)


def main():
    # === 1) IPRESS -> gdf_hospitales_clean ===
    gdf_hosp = read_ipress_as_points()

    # (opcional) GeoParquet si tu entorno lo soporta
    try:
        gdf_hosp.to_parquet(OUT_HOSP_GEO_PARQ, index=False)
        print("✔ GeoParquet hospitales:", OUT_HOSP_GEO_PARQ)
    except Exception as e:
        print("ℹ No se escribió GeoParquet de hospitales (opcional):", e)

    # Entregable oficial (tabular)
    save_parquet_tabular(gdf_hosp, OUT_HOSP_PARQ)

    # === 2) Leer DISTRITOS y sjoin ===
    gdf_dist = gpd.read_file(DIST_SHP)  # usa pyogrio por defecto
    gdf_dist = gdf_dist.to_crs("EPSG:4326") if gdf_dist.crs else gdf_dist.set_crs("EPSG:4326")

    # Campos típicos del shapefile (ajusta si tu shape trae otros)
    keep = [c for c in ["DEPARTAMEN", "PROVINCIA", "DISTRITO", "IDDIST", "IDPROV", "IDDPTO", "UBIGEO", "CAPITAL"]
            if c in gdf_dist.columns]
    gdf_dist_min = gdf_dist[keep + ["geometry"]].copy()

    # sjoin: punto dentro de polígono
    enr = gpd.sjoin(gdf_hosp, gdf_dist_min, how="left", predicate="within")
    enr = enr.rename(columns={k: f"DIST_{k}" for k in keep})

    # (opcional) GeoParquet
    try:
        enr.to_parquet(OUT_DIST_GEO_PARQ, index=False)
        print("✔ GeoParquet distritos_enriched:", OUT_DIST_GEO_PARQ)
    except Exception as e:
        print("ℹ No se escribió GeoParquet de distritos_enriched (opcional):", e)

    # Entregable oficial (tabular)
    save_parquet_tabular(enr, OUT_DIST_PARQ)

    # === 3) deptos_enriched a partir de IPRESS o del sjoin ===
    src_depto = ("Departamento" if "Departamento" in enr.columns
                 else "DEPARTAMENTO" if "DEPARTAMENTO" in enr.columns
                 else "DIST_DEPARTAMEN" if "DIST_DEPARTAMEN" in enr.columns
                 else None)
    if src_depto is None:
        raise ValueError("No encuentro columna de Departamento en IPRESS ni en el join (DIST_DEPARTAMEN).")

    deptos = enr.copy()
    deptos["DEPTO_NAME"] = deptos[src_depto].astype(str).str.strip()

    save_parquet_tabular(deptos, OUT_DEPTO_PARQ)

    print("\n✅ Entregables listos:")
    print("  -", OUT_HOSP_PARQ)
    print("  -", OUT_DIST_PARQ)
    print("  -", OUT_DEPTO_PARQ)


if __name__ == "__main__":
    main()


✔ IPRESS leído con encoding: latin1
✔ Deduplicado por 'Código Único': 7956 -> 7954
✔ GeoParquet hospitales: D:\Diplomado -\Python - intermedio\Tarea 2\gdf_hospitales_clean_geoparquet.parquet
✔ Parquet: D:\Diplomado -\Python - intermedio\Tarea 2\gdf_hospitales_clean.parquet
✔ GeoParquet distritos_enriched: D:\Diplomado -\Python - intermedio\Tarea 2\distritos_enriched_geoparquet.parquet
✔ Parquet: D:\Diplomado -\Python - intermedio\Tarea 2\distritos_enriched.parquet
✔ Parquet: D:\Diplomado -\Python - intermedio\Tarea 2\deptos_enriched.parquet

✅ Entregables listos:
  - D:\Diplomado -\Python - intermedio\Tarea 2\gdf_hospitales_clean.parquet
  - D:\Diplomado -\Python - intermedio\Tarea 2\distritos_enriched.parquet
  - D:\Diplomado -\Python - intermedio\Tarea 2\deptos_enriched.parquet
